<a href="https://colab.research.google.com/github/kurexi/ml-labs/blob/main/2_1_ForwardAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Forward Mode Automatic Differentiation

Forward mode AD can simply be implemented by defining a class to represent [dual numbers](https://en.wikipedia.org/wiki/Dual_number) which hold the value and its derivative. The following skeleton defines a dual number and implements multiplication.

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it?
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [45]:
import math

class DualNumber:
    def __init__(self, value, dvalue):
        self.value = value
        self.dvalue = dvalue

    def __str__(self):
        return str(self.value) + " + " + str(self.dvalue) + "ε"

    def __mul__(self, other):
        return DualNumber(self.value * other.value,
            self.dvalue * other.value + other.dvalue * self.value)

    def __add__(self, other):
        return DualNumber(self.value  + other.value,
                          self.dvalue + other.dvalue)

    def __truediv__(self, other):
        return DualNumber(self.value / other.value,
                          (self.dvalue * other.value - other.dvalue * self.value)/other.value**2)

    def __sub__(self, other):
        return DualNumber(self.value - other.value,
                          self.dvalue - other.dvalue)

    def __pow__(self, other):
        return DualNumber(self.value ** other.value,
                          other.value * self.value ** (other.value - 1) * self.dvalue)

In [6]:
# Tests

DualNumber(1,0) + DualNumber(1,0) / DualNumber(1,0) - DualNumber(1,0)**DualNumber(1,0)


## Implementing math functions

We also need to implement some core math functions. Here's the sine function for a dual number:

In [20]:
def sin(x):
    return DualNumber(math.sin(x.value), math.cos(x.value)*x.dvalue)

__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

In [39]:
# TODO: implement additional math functions on dual numbers

def cos(x):
    return DualNumber(math.cos(x.value), -math.sin(x.value)*x.dvalue)

def tan(x):
    return DualNumber(math.tan(x.value), x.dvalue/math.cos(x.value)**2)

def exp(x):
    return DualNumber(math.exp(x.value), math.exp(x.value)*x.dvalue)

In [17]:
# Tests
assert cos(DualNumber(0,0)).value == 1
assert tan(DualNumber(0,0)).value == 0
assert exp(DualNumber(0,0)).value == 1


## Time to try it out

We're now in a position to try our implementation.

__Task:__

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point.

In [29]:
def fn_z(x: DualNumber, y: DualNumber) -> DualNumber:
    return x * y + sin(x)

z = fn_z(DualNumber(0.5,1), DualNumber(4.2,0))
print(z)

2.579425538604203 + 5.077582561890373ε
2.1 + 4.2ε


__Task__: Differentiate the above function with respect to $x$ and write the symbolic derivatives in the following box. Verify the result computed above is correct by plugging-in the values into your symbolic gradient expression.

$$\begin{align}
\frac{\partial}{\partial{x}}\left[ x \cdot y + \sin(x) \right] &= y + \cos(x) \\
&=4.2 + \cos(0.5) \\
&= 5.077582561890373
\end{align}$$

__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above) and store the derivative in the variable `dzdy` (just the derivative, not the Dual Number). Verify by hand that the result is correct.

In [32]:
dzdy = fn_z(DualNumber(0.5,0), DualNumber(4.2,1)).dvalue

print('dz/dy:', dzdy)

dz/dy: 0.5


In [33]:
#Tests
assert dzdy
assert type(dzdy) == float


__Task:__ Finally, use the code block below to experiment and test the other math functions and methods you created.

In [47]:
def fn_sigmoid(x: DualNumber) -> DualNumber:
    return DualNumber(1,0) / (DualNumber(1,0)+exp(x*DualNumber(-1,0)))

def fn_power2(x: DualNumber) -> DualNumber:
    return x ** DualNumber(2,0)


print(fn_sigmoid(DualNumber(0,1)))
print(fn_sigmoid(DualNumber(1,1)))
print(fn_sigmoid(DualNumber(-1,1)))

print(fn_power2(DualNumber(0,1)))
print(fn_power2(DualNumber(1,1)))
print(fn_power2(DualNumber(-1,1)))

0.5 + 0.25ε
0.7310585786300049 + 0.19661193324148188ε
0.2689414213699951 + 0.19661193324148185ε
0 + 0ε
1 + 2ε
1 + -2ε
